# Введение в обработку текста на естественном языке

Материалы:
* Макрушин С.В. Лекция 9: Введение в обработку текста на естественном языке\
* https://realpython.com/nltk-nlp-python/
* https://scikit-learn.org/stable/modules/feature_extraction.html

## Задачи для совместного разбора

In [1]:
!pip install scikit-learn
!pip install pymorphy2

from sklearn.feature_extraction.text import CountVectorizer
import pymorphy2
from nltk.metrics.distance import edit_distance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 85.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13707 sha256=ef14296f6703e777282832dfe1551580b77aa64c56a973889e090cecab1ca47f
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt


1. Считайте слова из файла `litw-win.txt` и запишите их в список `words`. В заданном предложении исправьте все опечатки, заменив слова с опечатками на ближайшие (в смысле расстояния Левенштейна) к ним слова из списка `words`. Считайте, что в слове есть опечатка, если данное слово не содержится в списке `words`. 

In [ ]:
text = '''с велечайшим усилием выбравшись из потока убегающих людей Кутузов со свитой уменьшевшейся вдвое поехал на звуки выстрелов русских орудий'''

2. Разбейте текст из формулировки задания 1 на слова; проведите стемминг и лемматизацию слов.

In [2]:
!pip install nltk
from nltk.stem import SnowballStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
import nltk
nltk.download('punkt')


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

3. Преобразуйте предложения из формулировки задания 1 в векторы при помощи `CountVectorizer`.

## Лабораторная работа 9

### Расстояние редактирования

1.1 Загрузите предобработанные описания рецептов из файла `preprocessed_descriptions.csv`. Получите набор уникальных слов `words`, содержащихся в текстах описаний рецептов (воспользуйтесь `word_tokenize` из `nltk`). 

In [7]:
import pandas as pd
from nltk.tokenize import word_tokenize

In [8]:
preprocessed_descriptions = pd.read_csv("preprocessed_descriptions.csv")

words_set = set()
words_list = list()
words = [word_tokenize(item) for item in preprocessed_descriptions["preprocessed_descriptions"].to_list() if isinstance(item, str)]

[[words_set.add(x) for x in item] for item in words]
[[words_list.append(x) for x in item] for item in words]
    
print(f"Весего {len(words_list)} слов\nСреди них {len(words_set)} уникальных")

Весего 1069254 слов
Среди них 32868 уникальных


1.2 Сгенерируйте 5 пар случайно выбранных слов и посчитайте между ними расстояние редактирования.

In [10]:
import random
from nltk.metrics.distance import edit_distance

words_list = list(words_set)

for i in range(5):
    word1, word2 = random.sample(words_list, 2)
    print(f"Расстояние между '{word1}' и '{word2}': {edit_distance(word1, word2)}")

Расстояние между '201650' и 'seemingly': 9
Расстояние между 'exampleshes' и 'greentown': 11
Расстояние между 'moldy' и 'comfortinggoes': 12
Расстояние между 'foccacia' и 'opted': 7
Расстояние между 'cookbookserve' и 'edible': 11


1.3 Напишите функцию, которая для заданного слова `word` возвращает `k` ближайших к нему слов из списка `words` (близость слов измеряется с помощью расстояния Левенштейна)

In [11]:
from typing import Set, List

def same_words(word: str, k: int, words_data: Set[str]) -> List[str]:
    buf_tuple = [(edit_distance(word, item), item) for item in words_data]
    buf_tuple.sort(key=lambda x: x[0])
    return buf_tuple[:k]

same_words("seedless", 11, words_set)

[(0, 'seedless'),
 (1, 'needless'),
 (2, 'endless'),
 (2, 'needles'),
 (3, 'needle'),
 (3, 'eggless'),
 (3, 'settles'),
 (3, 'useless'),
 (3, 'seeded'),
 (3, 'seeds'),
 (3, 'feeders')]

### Стемминг, лемматизация

2.1 На основе результатов 1.1 создайте `pd.DataFrame` со столбцами: 
    * word
    * stemmed_word 
    * normalized_word 

Столбец `word` укажите в качестве индекса. 

Для стемминга воспользуйтесь `SnowballStemmer`, для нормализации слов - `WordNetLemmatizer`. Сравните результаты стемминга и лемматизации.

In [12]:
from nltk.stem import WordNetLemmatizer, SnowballStemmer
import nltk
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()
stemmer = SnowballStemmer('english')

[nltk_data] Downloading package wordnet to /root/nltk_data...


In [13]:
from nltk.stem import WordNetLemmatizer, SnowballStemmer

lemmatizer = WordNetLemmatizer()
stemmer = SnowballStemmer('english')

words_df = pd.DataFrame(words_set, columns=['word'])
words_df['stemmed_word'] = words_df['word'].apply(stemmer.stem)
words_df['normalized_word'] = words_df['word'].apply(lambda x: lemmatizer.lemmatize(x, pos='v'))

filtered_df = words_df[(words_df['word'] != words_df['normalized_word']) & (words_df['stemmed_word'] != words_df['normalized_word'])]

filtered_df = filtered_df.set_index('word')

filtered_df

,stemmed_word,normalized_word
word,,
scrounging,scroung,scrounge
indicates,indic,indicate
plunging,plung,plunge
glorified,glorifi,glorify
shook,shook,shake
...,...,...
applied,appli,apply
are,are,be
horses,hors,horse


2.2. Удалите стоп-слова из описаний рецептов. Какую долю об общего количества слов составляли стоп-слова? Сравните топ-10 самых часто употребляемых слов до и после удаления стоп-слов.

In [15]:
import nltk
nltk.download('stopwords')

import nltk
from nltk.corpus import stopwords
stopwords_set = set(stopwords.words('english'))

words_filtered = [item for item in words_list if item not in stopwords_set]
diff = round(len(words_filtered)/len(words_list)*100,2)
print(f"Всего слов: {len(words_list)}\nС удалением стоп-слов: {len(words_filtered)}\nДоля стоп-слов: {diff}%")

Всего слов: 32868
С удалением стоп-слов: 32732
Доля стоп-слов: 99.59%


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
#до_удаления
freq = nltk.FreqDist(words_list)
for word, number in freq.most_common(10):
    print(f"{number} -> {word}")

40072 -> the
34951 -> a
30245 -> and
26859 -> this
24836 -> i
23471 -> to
20285 -> is
19756 -> it
18364 -> of
15939 -> for


In [16]:
#после_удаления
freq = nltk.FreqDist(words_filtered)
for word, number in freq.most_common(10):
    print(f"{number} -> {word}")

1 -> sonia
1 -> billed
1 -> zmail
1 -> grated
1 -> didthis
1 -> maypo
1 -> unthaw
1 -> peassaffron
1 -> mello
1 -> chopping


### Векторное представление текста

3.1 Выберите случайным образом 5 рецептов из набора данных. Представьте описание каждого рецепта в виде числового вектора при помощи `TfidfVectorizer`

In [23]:
from sklearn.feature_extraction.text import TfidfVectorizer

# загрузка данных из файла
data = pd.read_csv('preprocessed_descriptions.csv')
data['preprocessed_descriptions']=data['preprocessed_descriptions'].apply(str)
# выбор 5 случайных рецептов
random_recipes = data.sample(5)

# создание объекта TfidfVectorizer
vectorizer = TfidfVectorizer()

# преобразование описания каждого рецепта в числовой вектор
for i, row in random_recipes.iterrows():
    description = row['preprocessed_descriptions']
    vector = vectorizer.fit_transform([description])
    print(f"Рецепт {i+1}:\n{description}\nВектор:\n{vector.toarray()}\n")

Рецепт 444:
this combination of dusky dates and tinted marzipan is a north african tradition worthy of a special occasion from cooking lightmay 2001
Вектор:
[[0.21320072 0.21320072 0.21320072 0.21320072 0.21320072 0.21320072
  0.21320072 0.21320072 0.21320072 0.21320072 0.21320072 0.21320072
  0.21320072 0.42640143 0.21320072 0.21320072 0.21320072 0.21320072
  0.21320072]]

Рецепт 13425:
this is an adaptation of sour cream banana bread which i posted then found out it was a duplicate the difference is you use yogurt instead of sour cream you may use plain or be adventurous and try vanilla lemon or your choice of flavors or go ahead and use sour cream if you wish i even used part mayonnaise once in a pinch for the nuts use whatever nuts you wish so here is the recipei hope you enjoy i used to make this for the colonial pines inn in highlands nc it was always well received and the customers could buy a loaf to take home which they frequently did
Вектор:
[[0.06835859 0.06835859 0.06835859

3.2 Вычислите близость между каждой парой рецептов, выбранных в задании 3.1, используя косинусное расстояние (`scipy.spatial.distance.cosine`) Результаты оформите в виде таблицы `pd.DataFrame`. В качестве названий строк и столбцов используйте названия рецептов.

In [24]:
import scipy
from sklearn.metrics.pairwise import cosine_distances

X = vectorizer.fit_transform(data['preprocessed_descriptions'])

matrix = X.toarray()

distances = cosine_distances(matrix[random_recipes.index])

result_df = pd.DataFrame(distances, index=random_recipes.index, columns=random_recipes.index)
result_df.columns.name = 'Recipes'
result_df.index.name = 'Recipes'
result_df

Recipes,443,13424,8429,15657,145
Recipes,,,,,
443,0.000000,0.974232,0.979179,0.983515,1.000000
13424,0.974232,0.000000,0.973174,0.808762,0.966270
8429,0.979179,0.973174,0.000000,0.962892,0.984047
15657,0.983515,0.808762,0.962892,0.000000,0.991802
145,1.000000,0.966270,0.984047,0.991802,0.000000


3.3 Какие рецепты являются наиболее похожими? Прокомментируйте результат (словами).